In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

# !git clone https://github.com/vikranth22446/assistant_intent_classifier
# !cd /content/drive
# !apt-get install git-lfs
# !mkdir GloVe
# !curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
# !unzip GloVe/glove.840B.300d.zip -d GloVe/

# !mkdir encoder
# !curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl

%cd /content/drive/My Drive/assistant_intent_classifier

import nltk
nltk.download('punkt')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/assistant_intent_classifier
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import json
import pandas as pd
import numpy as np
from collections import Counter
import subprocess
import sys
import matplotlib.pyplot as plt
import fileinput
from contextlib import closing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Add, Dropout, Concatenate, Embedding, Bidirectional, LSTM, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import torch
from models import InferSent
import nltk
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras import regularizers
import os
import datetime
np.random.seed(3252) 
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install contractions

     |████████████████████████████████| 317kB 7.2MB/s 
     |████████████████████████████████| 245kB 11.8MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81695 sha256=f6a6cd54f1011e89939aa7317d8b5197e807b88dccfcce59ec9b099d02ea2f96
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
import pandas
df_train = pandas.read_csv('df_train_squad_reminders.csv')
df_test = pandas.read_csv('df_test_squad_reminders.csv')


In [ ]:
import re
import contractions 

def get_reminders(df):
    reminders = []
    for index, row in df.iterrows():
        context = row['context'].replace('"', '').replace(".", ". ").replace("?", "? ").lower()
        processed = [contractions.fix(item).replace('.', "") for item in nltk.sent_tokenize(context)]
        reminders = reminders + nltk.sent_tokenize(context)
    return reminders

X_train = get_reminders(df_train)
y_train_one_hot = [[1, 0] for item in range(len(X_train))]

X_val = get_reminders(df_test)
y_val_one_hot = [[1, 0] for item in range(len(X_val))]

In [ ]:
X_train_reminder = X_train

In [ ]:
import json
import random
f = open('data_processed_42_intents.json',)
data = json.load(f)

train_oos = []
val_oos = []
for i in data:
    if i['intents'] == 'oos':
        if random.random() < 2 * len(X_train)/len(data):
            train_oos.append(i['text'])
        if random.random() < 2 * len(X_val)/len(data):
            val_oos.append(i['text'])

In [ ]:
X_train += train_oos
y_train_one_hot += [[0, 1] for item in range(len(train_oos))] 

X_val += val_oos
y_val_one_hot += [[0, 1] for item in range(len(val_oos))] 


In [ ]:
len(X_train), len(y_train_one_hot)

(4392, 4392)

In [ ]:
# Load model
%autoreload
def generate_Infersent_model():
    # Load model
    model_version = 1
    MODEL_PATH = "encoder/infersent%s.pkl" % model_version
    params_model = {'bsize': 64, 
                    'word_emb_dim': 300, 
                    'enc_lstm_dim': 2048,
                    'pool_type': 'max',
                    'dpout_model': 0.0, 
                    'version': model_version}
    model = InferSent(params_model)
    model.load_state_dict(torch.load(MODEL_PATH))

    W2V_PATH = 'GloVe/glove.840B.300d.txt'
    model.set_w2v_path(W2V_PATH)

    model.build_vocab_k_words(K=100000)

    return model

def get_doc2vec(text, model, verbose=False):
    emb = model.encode(text, verbose=verbose)
    return emb

def generate_basic_fully_connected():
    input_1 = Input((4096,), dtype=tf.float32)
    x = Dense(2000, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(input_1)
    
    x = Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
    x = Dropout(rate=0.5)(x)
    
    x = Dense(450, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
    x = Dropout(rate=0.5)(x)
    
    x = Dense(250, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
    out = Dense(2, activation='softmax')(x)
    dual_model = Model(inputs=input_1, outputs=out)
    
    adamOpti = Adam()
    dual_model.compile(optimizer=adamOpti, loss='categorical_crossentropy', metrics=['acc'])
    
    dual_model.summary()
    return dual_model

def get_cnn_model(num_output=140):
  # https://github.com/ajinkyaT/CNN_Intent_Classification/blob/master/Intent_Classification_Keras_Glove.ipynb
    MAX_SEQUENCE_LENGTH = 10 # Maximum number of words in a sentence
    MAX_NB_WORDS = 100000 # Vocabulary size
    EMBEDDING_DIM = 1000 # Dimensions of Glove word vectors 
    VALIDATION_SPLIT = 0.10

    filter_sizes = [2,3,5]
    num_filters = 512
    drop = 0.5

    inputs = Input(shape=(4096,), dtype='int32')
    # embedding = Embedding(input_dim=len(word_index) + 1, output_dim=EMBEDDING_DIM, weights=[embedding_matrix],
    #                       input_length=MAX_SEQUENCE_LENGTH, trainable=False)(inputs)
    # reshape = Reshape((MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,1))(inputs)

    conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], 4096), padding='valid', kernel_initializer='normal', activation='relu')(inputs)
    conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], 4096), padding='valid', kernel_initializer='normal', activation='relu')(inputs)
    conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], 4096), padding='valid', kernel_initializer='normal', activation='relu')(inputs)

    maxpool_0 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
    maxpool_1 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
    maxpool_2 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

    concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
    flatten = Flatten()(concatenated_tensor)
    dropout = Dropout(drop)(flatten)
    preds = Dense(num_output, activation='softmax')(dropout)
    adamOpti = Adam()
    preds.compile(optimizer=adamOpti, loss='categorical_crossentropy', metrics=['acc'])

    preds.summary()
    return preds

In [ ]:
%%time
model = generate_Infersent_model()
# model = torch.load('infraset_model.torch')
model.eval()


Vocab size : 100000
CPU times: user 6.43 s, sys: 751 ms, total: 7.19 s
Wall time: 7.42 s


In [ ]:

X_train_vec = get_doc2vec(X_train, model)
X_val_vec = get_doc2vec(X_val, model)


In [ ]:
np.save('X_train_reminders_vec.npy', X_train_vec) # save
np.save('X_val_reminders_vec.npy', X_val_vec) # save

In [ ]:
y_train_one_hot = np.array(y_train_one_hot)
y_val_one_hot = np.array(y_val_one_hot)

In [ ]:

ffcc = generate_basic_fully_connected()
ffcc.fit(X_train_vec, y_train_one_hot, 
         epochs=100, batch_size=32, shuffle=True,
         validation_data=(X_val_vec, y_val_one_hot)
         , callbacks = [EarlyStopping(monitor='val_loss', patience=12)])
ffcc.save('fcc_keras_reminders_model')

In [ ]:
def get_label_decoded(index):
    return ["reminder", "oos"][index]

def get_labels_decoded(labels):
    results = np.argmax(labels, axis=1)
    decoded = []
    for label in results:
        label = get_label_decoded(label)
        decoded.append(label)
    return decoded

def calculate_accuracy(model, X_vec, y_labels, nueral_net=True, with_oos=True, ignore_correct=True, cutoff=0.7):
    predictions = model.predict(X_vec)
    predicted_labels = get_labels_decoded(predictions) # each integer is [0, 0, 0,1]
    posterior_prob = np.max(predictions, axis=1)
        # posterior_prob =  
    correct = 0
#     classification = {key:defaultdict(int) for key in y_labels}
    classification = []
    num_oos = 0
    for pred,y_true, prob in zip(predicted_labels, y_labels, posterior_prob):
        # if prob < cutoff:
        #     pred = 'oos'
        if y_true == 'oos' and not with_oos:
          num_oos += 1
          continue
        if pred == y_true:
            correct += 1
            if ignore_correct:
              continue
        classification.append({"y_true": y_true, "pred": pred, "prob": prob})

    classification = pd.DataFrame(classification)
    classification = classification.groupby(['y_true','pred']).agg({'prob':[('prob', 'mean')], 'pred': [('count','count')]}).reset_index()
    classification.columns = ["y_true", "pred", "prob", "count"]
    print(classification)

    accuracy = correct/(len(predictions) - num_oos)
    
    return accuracy, classification

In [ ]:
y_train_labels = get_labels_decoded(y_train_one_hot)
y_val_labels = get_labels_decoded(y_val_one_hot)

In [ ]:
def analyze_missclassifications(classifications):
  dic = {}
  for index, row in classifications.iterrows():
      y_true = row['y_true']
      pred = row['pred']
      posterior_prob = row['prob']
      count = row['count']
      if y_true not in dic:
        dic[y_true] = {"count": 0}
      dic[y_true]["count"] += count
  
      if y_true == pred:
        dic[y_true]["true count"] = count
        dic[y_true]["confidence"] = posterior_prob
  l = []
  for key, val in dic.items():
      if "true count" not in val:
        dic[key]["true count"] = 0
        dic[key]["confidence"] = 0.0
      dic[key]["TP"] = dic[key]["true count"]/dic[key]["count"]
      l.append({"y_true": key, "confidence": dic[key]["confidence"], "count": dic[key]["count"], "TP": dic[key]["TP"]})
  res = pd.DataFrame(l).sort_values(['confidence', 'TP', 'count'],ascending=False)
  return res

In [ ]:
%%time
train_accuracy, train_classifications = calculate_accuracy(ffcc, X_train_vec, y_train_labels, ignore_correct=False, with_oos=True)
print("Train Accuracy With OOS", train_accuracy)


     y_true      pred      prob  count
0       oos       oos  0.999693   2275
1  reminder       oos  0.879694      1
2  reminder  reminder  0.999871   2021
Train Accuracy With OOS 0.9997672794973237
CPU times: user 227 ms, sys: 27.3 ms, total: 254 ms
Wall time: 208 ms


In [ ]:
%%time
validation_accuracy, validation_classifications = calculate_accuracy(ffcc, X_val_vec, y_val_labels, ignore_correct=False, with_oos=True)
print("Validation Accuracy With OOS", validation_accuracy)


     y_true      pred      prob  count
0       oos       oos  0.999464    241
1       oos  reminder  0.998382      1
2  reminder       oos  0.974393      4
3  reminder  reminder  0.999044    226
Validation Accuracy With OOS 0.989406779661017
CPU times: user 63.1 ms, sys: 1.04 ms, total: 64.1 ms
Wall time: 57.2 ms


In [ ]:
def quick_predict_label(infraset_model, classification_model, text, cutoff=0.0):
    X = get_doc2vec([text], infraset_model)
    predictions = classification_model.predict(X)
    predicted_labels = get_labels_decoded(predictions) # each integer is [0, 0, 0,1]
    prob = np.max(predictions, axis=1) # each integer is [0, 0, 0,1]
    if prob < cutoff:
      predicted_labels[0] = 'oos' # since only one item
    return predicted_labels, prob


In [ ]:
quick_predict_label(model, ffcc, "At what time is the dataset done")

(['oos'], array([0.99995685], dtype=float32))

In [ ]:
quick_predict_label(model, ffcc, "Reminder me to go to the Park")

(['reminder'], array([1.], dtype=float32))

In [ ]:
!pip install padatious

  Using cached https://files.pythonhosted.org/packages/d0/e7/70a6eb34b7e67fef5b2645df2ee1f807db2b5a345e4e6adfb2660a56425b/padatious-0.4.8.tar.gz
  Using cached https://files.pythonhosted.org/packages/80/a1/fed455d25c34a62d4625254880f052502a49461a5dd1b80854387ae2b25f/fann2-1.1.2.tar.gz
     |████████████████████████████████| 225kB 9.6MB/s 
  Created wheel for padatious: filename=padatious-0.4.8-py2.py3-none-any.whl size=24603 sha256=5cdee3d71f0c0c5777f1dba4d35949fcbd3f42e7749b8c21bbfd91bf6f063046
  Stored in directory: /root/.cache/pip/wheels/17/aa/ed/5dbb2bfbbabf2d13ffaa4b4e3bda2f8f97e621df189461f978
  Created wheel for fann2: filename=fann2-1.1.2-cp36-cp36m-linux_x86_64.whl size=299043 sha256=18a5353e946d81d4620c030b21bab6a0ad7624cdca5be033040164e13d9ec865
  Stored in directory: /root/.cache/pip/wheels/1b/90/e3/b97a2310a956f17c56048801aabedafcffbee48ff62d0374ed
  Created wheel for padaos: filename=padaos-0.1.10-cp36-none-any.whl size=3150 sha256=3a6c5f2b8083b1251be69f246a1e12b637250ca

In [ ]:
from padatious import IntentContainer

container = IntentContainer('intent_cache')
container.add_intent('reminder', X_train_reminder)
# container.add_intent('oos', [item.replace(".", " ").replace(":", " ").replace(")", " ").replace("(", " ").replace("!", " ").replace(";", " ").replace(",", " ") for item in train_oos])
# container.add_intent('search', ['Search for {query} (using|on) {engine}.'])
container.train()

print(container.calc_intent('Remind me to go to the park tommorow afternoon with the kids'))
# print(container.calc_intent('At what time with the data be finished'))

# container.remove_intent('goodbye')

{'name': 'reminder', 'sent': 'remind me to go to the park tommorow afternoon with the kids', 'matches': {}, 'conf': 0.6059698669580462}


In [ ]:
len(X_train_reminder)

4320

/content


Reading package lists... Done
Building dependency tree       
Reading state information... Done
libfann-dev is already the newest version (2.2.0+ds-3).
swig is already the newest version (3.0.12-1).
python3-dev is already the newest version (3.6.7-1~18.04).
python3-pip is already the newest version (9.0.1-2.3~ubuntu1.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.


In [ ]:
!

Scanning dependencies of target floatfann
[  4%] Building C object src/CMakeFiles/floatfann.dir/floatfann.c.o
[  8%] Linking C shared library libfloatfann.so
[  8%] Built target floatfann
Scanning dependencies of target floatfann_static
[ 12%] Building C object src/CMakeFiles/floatfann_static.dir/floatfann.c.o
[ 16%] Linking C static library libfloatfann.a
[ 16%] Built target floatfann_static
Scanning dependencies of target doublefann
[ 20%] Building C object src/CMakeFiles/doublefann.dir/doublefann.c.o
[ 24%] Linking C shared library libdoublefann.so
[ 24%] Built target doublefann
Scanning dependencies of target doublefann_static
[ 28%] Building C object src/CMakeFiles/doublefann_static.dir/doublefann.c.o
[ 32%] Linking C static library libdoublefann.a
[ 32%] Built target doublefann_static
Scanning dependencies of target fixedfann
[ 36%] Building C object src/CMakeFiles/fixedfann.dir/fixedfann.c.o
[ 40%] Linking C shared library libfixedfann.so
[ 40%] Built target fixedfann
Scanning d